## Install Requirements

In [1]:
"""# Cell 1: Install Requirements
!pip install backtrader==1.9.78.123 pyfolio==0.9.2 empyrical==0.5.5 
!pip install pandas_ta==0.3.14b yfinance==0.2.37
!pip install -U numpy==1.24.3  # Critical for compatibility
!pip install backtrader pyfolio empyrical matplotlib pandas_ta
!pip install -U numpy  # Ensure numpy >= 1.24.3"""

'# Cell 1: Install Requirements\n!pip install backtrader==1.9.78.123 pyfolio==0.9.2 empyrical==0.5.5 \n!pip install pandas_ta==0.3.14b yfinance==0.2.37\n!pip install -U numpy==1.24.3  # Critical for compatibility\n!pip install backtrader pyfolio empyrical matplotlib pandas_ta\n!pip install -U numpy  # Ensure numpy >= 1.24.3'

## Imports

In [2]:
import pandas as pd
import numpy as np
import backtrader as bt
import pyfolio as pf
from datetime import datetime
import matplotlib.pyplot as plt
import pandas_ta as ta
plt.style.use('ggplot')
%matplotlib inline

## Data Preparation

In [3]:
# Load and Filter Dataset
def load_trading_data():
    df = pd.read_csv('trading_dataset.csv', 
                    index_col=0, 
                    parse_dates=True)
    
    # Filter to valid trading days
    df = df[df.index >= pd.to_datetime('2010-01-04')]
    df = df[df.index <= pd.to_datetime(datetime.today().strftime('%Y-%m-%d'))]
    
    # Forward-fill missing VIX data (market closed days)
    df['VIXCLS'].ffill(inplace=True)
    
    return df

full_data = load_trading_data()
print(f"Data Range: {full_data.index[0].date()} to {full_data.index[-1].date()}")
display(full_data.tail(3))

Data Range: 2010-02-09 to 2025-03-21


,SPY_Open,SPY_High,SPY_Low,SPY_Close,SPY_Volume,SPY_RSI_14,SPY_MACD_12_26,SPY_BBL_20_2,SPY_BBU_20_2,SPY_ATR_14,...,QQQ_High,QQQ_Low,QQQ_Close,QQQ_Volume,QQQ_RSI_14,QQQ_MACD_12_26,QQQ_BBL_20_2,QQQ_BBU_20_2,QQQ_ATR_14,VIXCLS
2025-03-19,561.141986,569.237628,559.945573,565.429077,66556000,42.474441,-9.986654,542.971570,608.839545,9.842078,...,484.607311,474.162895,480.173950,40430300,40.697011,-11.997114,455.716888,532.963727,11.189526,19.90
2025-03-20,561.640511,568.858788,560.912659,563.794006,62958200,41.600824,-9.475738,543.988988,603.346590,9.706653,...,483.828457,475.470925,478.546356,36780500,39.944797,-11.515770,457.477834,525.414417,10.987240,19.80
2025-03-21,559.280029,564.890015,558.030029,563.979980,83763000,41.747584,-8.952627,544.244272,599.675234,9.503320,...,480.892835,472.205809,480.123993,42234900,41.081575,-10.881563,458.693134,519.681853,10.822939,19.28


##  Backtrader Data Feed Configuration

In [4]:
# Custom Data Feed Class
class MultiTickerDataFeed(bt.feeds.PandasData):
    lines = (
        'rsi', 'macd', 'bbl', 'bbu', 
        'atr', 'vix', 'spread'
    )
    
    params = (
        ('datetime', None),
        ('open', 'SPY_Open'),
        ('high', 'SPY_High'),
        ('low', 'SPY_Low'),
        ('close', 'SPY_Close'),
        ('volume', 'SPY_Volume'),
        ('openinterest', -1),
        ('rsi', 'SPY_RSI_14'),
        ('macd', 'SPY_MACD_12_26'),
        ('bbl', 'SPY_BBL_20_2'),
        ('bbu', 'SPY_BBU_20_2'),
        ('atr', 'SPY_ATR_14'),
        ('vix', 'VIXCLS'),
        ('spread', -1)  # Placeholder for future use
    )

def create_data_feed(ticker: str):
    """Dynamically creates data feed for any ticker"""
    # Get parameter names from the class definition
    param_names = [p[0] for p in MultiTickerDataFeed.params]
    
    # Create column mapping
    col_map = {
        f: f.replace('SPY', ticker)
        for f in param_names
        if 'SPY' in f and f != 'datetime'
    }
    
    return MultiTickerDataFeed(
        dataname=full_data,
        name=ticker,
        **col_map
    )

## Trading Strategy Implementation

In [5]:
# Enhanced Mean Reversion Strategy
class VIXAwareMeanReversion(bt.Strategy):
    params = (
        ('rsi_period', 14),
        ('vix_entry', 18),
        ('vix_exit', 25),
        ('position_cap', 0.3),  # Max 30% per position
        ('slippage', 0.0005),   # 5bps
        ('trade_delay', 2)      # Days to wait after signal
    )

    def __init__(self):
        # Trackers for each ticker
        self.trade_states = {
            'SPY': {'active': False, 'entry_date': None},
            'QQQ': {'active': False, 'entry_date': None}
        }
        
        # Portfolio constraints
        self.max_concurrent_trades = 2
        self.portfolio_risk_limit = 0.2  # 20% max drawdown
        
        # Indicators cross-reference
        self.data_close = {data._name: data.close for data in self.datas}
        self.data_rsi = {data._name: data.rsi for data in self.datas}
        self.data_vix = self.datas[0].vix  # VIX is same for all

    def next(self):
        current_date = self.datetime.date()
        cash = self.broker.get_cash()
        portfolio_value = self.broker.getvalue()
        
        # Risk check - pause trading if beyond drawdown limit
        if self.analyzers.drawdown.get_analysis()['drawdown'] > self.portfolio_risk_limit:
            return
        
        for data in self.datas:
            ticker = data._name
            position = self.getposition(data)
            price = data.close[0]
            rsi = self.data_rsi[ticker][0]
            vix = self.data_vix[0]
            
            # Calculate position size with slippage buffer
            risk_cash = cash * self.params.position_cap
            size = int((risk_cash / price) * (1 - self.params.slippage))
            
            # Entry Logic
            if not position and not self.trade_states[ticker]['active']:
                if (rsi < 30 and 
                    vix < self.params.vix_entry and
                    data.macd[0] > data.macd[-1]):
                    
                    # Submit delayed order
                    self.trade_states[ticker] = {
                        'active': True,
                        'entry_date': current_date
                    }
                    self.buy(
                        data=data,
                        size=size,
                        exectype=bt.Order.Limit,
                        price=price * (1 - self.params.slippage),
                        valid=current_date + pd.DateOffset(days=self.params.trade_delay)
                    )
                    
            # Exit Logic        
            elif position and self.trade_states[ticker]['active']:
                days_in_trade = (current_date - self.trade_states[ticker]['entry_date']).days
                
                if (rsi > 70 or 
                    vix > self.params.vix_exit or
                    days_in_trade > 14):
                    
                    self.close(data=data)
                    self.trade_states[ticker]['active'] = False

    def notify_order(self, order):
        """Handle order execution failures"""
        if order.status in [order.Expired, order.Margin]:
            self.trade_states[order.data._name]['active'] = False

## Verify Parameter Structure

In [7]:
# Add this diagnostic cell before running the backtest
print("DataFeed Parameters:")
for p in MultiTickerDataFeed.params:
    print(f"Name: {p[0]}, Default: {p[1]}")

# Should output:
# Name: datetime, Default: None
# Name: open, Default: SPY_Open
# ... etc ...

DataFeed Parameters:


TypeError: 'type' object is not iterable

## Backtest Execution

In [6]:
# Cell 6: Configure and Run Backtest
def run_complete_backtest():
    cerebro = bt.Cerebro(stdstats=False, optreturn=False)
    
    # 1. Add Data Feeds
    cerebro.adddata(create_data_feed('SPY'))
    cerebro.adddata(create_data_feed('QQQ'))
    
    # 2. Add Strategy
    cerebro.addstrategy(VIXAwareMeanReversion)
    
    # 3. Configure Broker
    cerebro.broker.setcash(1000000.0)  # $1M initial capital
    cerebro.broker.setcommission(
        commission=0.001,  # 0.1% per trade
        leverage=1.0,
        margin=0.0
    )
    
    # 4. Add Slippage Model
    cerebro.broker.set_slippage_perc(
        perc=0.0005,  # 5bps
        slip_open=True,
        slip_limit=True,
        slip_match=True
    )
    
    # 5. Add Analyzers
    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.01, annualize=True)
    cerebro.addanalyzer(bt.analyzers.DrawDown)
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer)
    
    # 6. Run Backtest
    print('=== BACKTEST START ===')
    results = cerebro.run()
    print('=== BACKTEST COMPLETE ===')
    
    return cerebro, results[0]

# Execute (may take 5-15 minutes)
cerebro, result = run_complete_backtest()

AttributeError: type object 'AutoInfoClass_LineRoot_LineMultiple_LineSeries_Dat' has no attribute '_keys'